# Projeto Final - Aprendizagem de Máquina
- Thiago Martin Poppe
- Matrícula: 2017014324

# Introdução ao dataset
- Para esse projeto, escolhi utilizar o dataset ``Traditional Food around the world`` presente no site [Kaggle](https://www.kaggle.com/abhijeetbhilare/world-cuisines). O dataset é formado por diversas imagens que demonstram pratos culinários tradicionais de diversas regiões do mundo, como por exemplo ``American``, ``European`` e ``Indian``.

# Objetivo e ideias
- O objetivo desse projeto é dado um prato determinar o país de origem do mesmo. Além dessa classificação, usaremos explicabilidade para tentar entender o motivo do modelo ter aprendido que um prato X pertence à região Y. Também iremos comparar diversas arquiteturas propostas para resolver o problema, como por exemplo MLP's, LeNet-5, entre outras.<br><br>

In [1]:
import os
import cv2
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

from tqdm import tqdm
from torch import nn

import warnings
warnings.filterwarnings("ignore") # filtering depricated warnings
np.random.seed(42) # fixing random seed to 42

# Definições de constantes

- Nessa célula definimos "constantes" que utilizaremos ao longo do notebook, como por exemplo: diretório raiz que contém as pastas com as imagens, as classes que usaremos, etc.

In [2]:
ROOT_DIR = "Dishes/"
IMG_SIZE = 128

N_CLASSES = 6
CLASSES = {
    "American": 0, "Chinese": 1,
    "European": 2, "Indian":  3,
    "Japanese": 4, "Korean":  5
}

# Funções de leitura e exibição de imagens

- A função de leitura utiliza a biblioteca ``OpenCv`` para ler, escalar e converter o mapeamento de cores para RGB, visto que, originalmente, a biblioteca lê a imagem com o mapeamento BGR (blue, green, red). Além disso, a função normaliza os valores dos pixels dividindo os mesmos por 255, que corresponde ao valor máximo de um pixel.<br><br>

- A função de exibição utiliza a biblioteca ``MatplotLib`` para exibir as imagens. Aqui não temos nada muito complexo, sendo apenas uma chamada de função já implementada na própria biblioteca.

In [3]:
def read_img(filename):
    """
        Helper function to read an image
        
        - Parameters:
            filename : string
        - Return:
            A RGB image
    """
    
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # by default opencv uses BGR
    
    return np.array(img) / 255.0

In [4]:
def show_img(img, cmap=None):
    """
        Helper function to plot/show image
        
        - Parameters:
            img : Image
            cmap : Grayscale, RGB, etc
    """
    
    plt.imshow(img, cmap=cmap)
    plt.show()

# Representação one-hot vector
- A representação one-hot consiste em termos um vetor composto por 0's e apenas uma posição com valor 1. Utilizamos esse artifício para representar cada classe do nosso dataset.<br><br>

- Em um mundo ideal, queremos que a classificação tenha 100% de certeza, i.e que a resposta do classificador seja um vetor contendo apenas um valor 1 e os demais 0. Porém, na prática, isso não irá ocorrer sempre.

In [5]:
def one_hot(label):
    """
        Helper function that creates an one-hot vector given label
        
        - Parameters:
            label: string
            
        - Return:
            An one-hot vector
    """
    
    return np.eye(N_CLASSES)[CLASSES[label]]

In [6]:
for label in CLASSES:
    print(f"Class {label} will be seen as {one_hot(label)}")

Class American will be seen as [1. 0. 0. 0. 0. 0.]
Class Chinese will be seen as [0. 1. 0. 0. 0. 0.]
Class European will be seen as [0. 0. 1. 0. 0. 0.]
Class Indian will be seen as [0. 0. 0. 1. 0. 0.]
Class Japanese will be seen as [0. 0. 0. 0. 1. 0.]
Class Korean will be seen as [0. 0. 0. 0. 0. 1.]


# Leitura dos dados
- Nessa célula realizamos a leitura dos dados. Cada classe possui sua própria pasta contendo as imagens relacionadas com a comida tradicional da região.<br><br>

- Após a leitura e armazenamento de todos os dados em um DataFrame do ``pandas`` nós realizamos um shuffle para não termos futuros problemas com o aprendizado.

In [7]:
imgs = []
labels = []
counter = {label:0 for label in CLASSES}
    
for label in CLASSES:
    print(f"Loading {label} dishes...")        
    path = os.path.join(ROOT_DIR, label)
    for f in tqdm(os.listdir(path)):
        filename = os.path.join(path, f)

        try: # some images may be corrupted
            imgs.append(read_img(filename))
            labels.append(label)
            data.append([X, y])
            counter[label] += 1

        except Expection as E:
            print(str(E))

np.random.shuffle(data)

Loading American dishes...


100%|█████████████████████████████████████████████████████████████████████████████| 1815/1815 [00:06<00:00, 285.90it/s]


Loading Chinese dishes...


  0%|                                                                                         | 0/1994 [00:00<?, ?it/s]

Loading European dishes...


...

  0%|                                                                                         | 0/2065 [00:00<?, ?it/s]

Loading Japanese dishes...


  0%|                                                                                         | 0/2422 [00:00<?, ?it/s]

Loading Korean dishes...


100%|█████████████████████████████████████████████████████████████████████████████| 2422/2422 [00:08<00:00, 276.70it/s]


# Balanceamento de classes

- Como podemos ver, temos uma distribuição balanceada das classes. A maior diferença de dados se encontra entre as classes ``American`` e ``Chinese``, onde a segunda possui $\approx 1,67$ vezes mais imagens que a primeira.

In [8]:
for label in counter:
    print(f"{label} class has {counter[label]} images")

American class has 1815 images
Chinese class has 3030 images
European class has 1994 images
Indian class has 2453 images
Japanese class has 2065 images
Korean class has 2422 images


# Separação entre imagens e classes

In [ ]:
X = torch.Tensor([i[0] for i in data]).view(-1, 3, IMG_SIZE, IMG_SIZE)
y = torch.Tensor([i[1] for i in data])

# Arquiteturas utilizadas

In [9]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self._input_size  = input_size
        self._hidden_size = hidden_size
        
        # Defining MLP architecture
        self._layers = nn.Sequential(
            # Input
            nn.Linear(self._input_size, self._hidden_size),
            nn.ReLU(),
            
            # Hidden Layer
            nn.Linear(self._hidden_size, N_CLASSES),
            nn.Softmax()
        )

    def forward(self, x):
        return self._layers(x)